In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time as systime
import datetime as dtime
import string
import seaborn as sns
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split #for splitting the training set
from sklearn.metrics import log_loss #evaluation metric
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

%pylab inline

from IPython.core.display import HTML
def css_styling():
    styles = open("style.css", "r").read()
    return HTML(styles)
css_styling()

Populating the interactive namespace from numpy and matplotlib


/Library/Python/2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
def normalize(data): #feature normalization
    data = (data - data.mean()) / (data.max() - data.min())
    return data

num2month={1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'jun',7:'jul',8:'aug',9:'sep',10:'oct',11:'nov',12:'dec'}

In [3]:
train=pd.read_csv('data/train.csv', parse_dates = ['Dates'])
test=pd.read_csv('data/test.csv', parse_dates = ['Dates'])

#train.info()

In [4]:
def prep_data(data, istest):
    if(istest==0):
        #Convert crime labels to numbers
        le_crime = preprocessing.LabelEncoder()
        crime = le_crime.fit_transform(data.Category)
 
    #Get binarized weekdays, districts, and hours.
    days = pd.get_dummies(data.DayOfWeek)
    district = pd.get_dummies(data.PdDistrict)
    month = pd.get_dummies(data.Dates.dt.month.map(num2month))
    hour = data.Dates.dt.hour
    hour = pd.get_dummies(hour) 
 
    #Build new array
    new_data = pd.concat([hour, month, days, district], axis=1)
    new_data['X']=normalize(data.X)
    new_data['Y']=normalize(data.Y)
    #train_data['hour']=normalize(train.Dates.dt.hour)
    if(istest==0):
        new_data['crime']=crime
        
    new_data['dark'] = data.Dates.dt.hour.apply(lambda x: 1 if (x >= 18 or x < 6) else 0)
    return new_data

train_proc = prep_data(train,0)
test_proc = prep_data(test,1)

In [16]:
features = [#'hour',
            #'dark',
            'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec',
            'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 
            'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION','NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN',
            #'X','Y'
           ] + [x for x in range(0,24)]
 
training, validation = train_test_split(train_proc, train_size=.67)
model = BernoulliNB()
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['crime'], predicted) 

2.5844812045165786

In [12]:
model = BernoulliNB()
model.fit(train_proc[features], train_proc['crime'])
predicted = model.predict_proba(test_proc[features])
 

In [17]:
#Write results
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(train.Category)
result=pd.DataFrame(predicted, columns=le_crime.classes_)
result.to_csv('submit.csv', index = True, index_label = 'Id' )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884262 entries, 0 to 884261
Data columns (total 39 columns):
ARSON                          884262 non-null float64
ASSAULT                        884262 non-null float64
BAD CHECKS                     884262 non-null float64
BRIBERY                        884262 non-null float64
BURGLARY                       884262 non-null float64
DISORDERLY CONDUCT             884262 non-null float64
DRIVING UNDER THE INFLUENCE    884262 non-null float64
DRUG/NARCOTIC                  884262 non-null float64
DRUNKENNESS                    884262 non-null float64
EMBEZZLEMENT                   884262 non-null float64
EXTORTION                      884262 non-null float64
FAMILY OFFENSES                884262 non-null float64
FORGERY/COUNTERFEITING         884262 non-null float64
FRAUD                          884262 non-null float64
GAMBLING                       884262 non-null float64
KIDNAPPING                     884262 non-null float64
LARCENY/T